## 🧑‍💻 Welcome to the OpenAI Agents SDK Introduction!

In this session, we'll explore the **OpenAI Agents SDK** and its core components:

### What We'll Cover

- **Agents**  
  > An *agent* is an LLM (Large Language Model) equipped with tools and memory, capable of performing any task or delegating tasks to other agents.

- **Runner**  
  > The *Runner* is a helper class that assists in executing an agent loop.

- **`function_tool`**  
  > The `function_tool` is a decorator that allows you to use any Python function as a tool, leveraging the function's name, parameters, and docstring.

- **Structured Outputs with Pydantic**  
  > You can define structured outputs for your agents using [Pydantic's](https://docs.pydantic.dev/) `BaseModel` and `Field`. This allows you to specify the exact format and types of the agent's responses, making it easier to validate and use the outputs in your applications.

### Prerequisite: Install the OpenAI Agents SDK

Before you begin, make sure you have the OpenAI Agents SDK installed.
If you're using [OpenAI's `uv` package manager](https://github.com/astral-sh/uv), run:

```bash
uv add openai-agents
```

If you're using `pip`, you can run:

```bash
pip install openai-agents
```

Now you're ready to get started!


Let's get started!

In [ ]:
# import the necessary libraries

from agents import Agent, Runner, function_tool
from pydantic import BaseModel, Field

### Why do we need `nest_asyncio` in Python notebooks?

Jupyter notebooks (and other interactive Python environments) already run an event loop in the background to support features like code execution and output display. However, many modern Python libraries—including the OpenAI Agents SDK—use `asyncio` to manage asynchronous tasks.

Normally, you can't start a new event loop if one is already running, which leads to errors like `RuntimeError: This event loop is already running`. The `nest_asyncio` library patches the event loop so that you can safely run asynchronous code (including `async` functions and libraries that use `asyncio`) inside a notebook cell, even if an event loop is already active.

In summary:  
**`nest_asyncio` allows you to run asynchronous code in Jupyter notebooks without event loop conflicts.**


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Build your first agent using the OpenAI Agents SDK's Agent class. Provide the following parameters:
# - name: "My First Agent"
# - instructions: "You are a helpful assistant/agent who answers users' queries."
# - model: "gpt-4o-mini"

my_first_agent = Agent(
    name="My First Agent",
    instructions="You are a helpful assistant/agent who answers users' queries.",
    model="gpt-4o-mini"
)

In [ ]:
# Run the agent loop using the Runner class. Provide the following parameters:
# - starting_agent: The agent to start the loop with.
# - input: The input to pass to the agent.

# Print the final output

agent_output = Runner.run_sync(starting_agent=my_first_agent, input="What is the capital of France?")
print(agent_output.final_output)

In [ ]:
# Let's build an agent that uses dummy weather data to help you decide whether to go out or not.
# Define a dummy weather tool, pydantic output schema and weather agent
# Don't forget to use final_output_as() with a pydantic schema

class WeatherAgentOutput(BaseModel):
    temperature: int = Field(description="The temperature in Celsius")
    should_go_out: bool = Field(description="Whether you should go out or not")
    reasoning: str = Field(description="The reasoning for the answer")


@function_tool
def get_weather(city: str):
    """
    Get the weather for a city.

    Args:
        city (str): The city to get the weather for.
    """
    return {"temperature": 20, "unit": "Celsius", "description": "sunny"}

weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a weather agent. You answer questions about the weather.",
    model="gpt-4o-mini",
    tools=[get_weather],
    output_type=WeatherAgentOutput
)

result = await Runner.run(starting_agent=weather_agent, input="Depending on the weather, should I go to Tokyo?")
print(result.final_output_as(WeatherAgentOutput))

In [ ]:
# Let's build another agent, Math Tutor agent that solves math expressions and provides steps.
# Define a pydantic output schema for the Math Tutor agent and a tool for defining agent's output

class MathTutorOutput(BaseModel):
    final_answer: int = Field(description="The final answer to the question")
    steps: list[str] = Field(description="The steps to solve the problem")

@function_tool
def solve_math_expression(expression: str):
    """
    Solve a math expression.

    Args:
        expression (str): The math expression to solve.
    """
    print(f"Solving math expression: {expression}")
    return eval(expression)

math_tutor_agent = Agent(
    name="Math Tutor",
    instructions="You are a math tutor. You solve math expressions and provide steps. If a question is out of scope of mathematics, you say so. You do not calculate the answer to the question on your own. Instead you use the tools provided.",
    model="gpt-4o-mini",
    tools=[solve_math_expression],
    output_type=MathTutorOutput,
)

result = await Runner.run(starting_agent=math_tutor_agent, input="What is the circumference of a circle with radius 5?")
print(result.final_output_as(MathTutorOutput))